In [0]:
# ======================================================================================
# NOTEBOOK DE TRANSFORMATION : DimCustomer
# ======================================================================================
import logging
import sys
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, monotonically_increasing_id

# Ajoute le dossier 'src' au chemin pour pouvoir importer notre bibliothèque
sys.path.append('../src') 
import common_utils # Importe notre bibliothèque partagée

# ======================================================================================
# 1. DÉCLARATION DES PARAMÈTRES (WIDGETS)
# ======================================================================================
dbutils.widgets.text("storage_account", "stsalesinsightcuxm0611", "Nom du compte de stockage")
dbutils.widgets.get("storage_account").strip()
dbutils.widgets.text("container", "data", "Nom du conteneur")
dbutils.widgets.text("silver_folder", "silver/sales_orders/", "Dossier source dans la couche Silver")
dbutils.widgets.text("secret_scope", "dbricks-scope-projet", "Scope unique pour les secrets du projet")
dbutils.widgets.text("adls_secret_key", "adls-access-key", "Clé du secret pour l'accès ADLS")
dbutils.widgets.text("sql_user_key", "sql-admin-user", "Clé du secret pour l'utilisateur SQL")
dbutils.widgets.text("sql_password_key", "sql-admin-password", "Clé du secret pour le mot de passe SQL")
dbutils.widgets.text("jdbc_hostname", "sqlsvr-salesinsightcuxm0611.database.windows.net", "Serveur Azure SQL DB")
dbutils.widgets.text("jdbc_database", "sqldb-salesinsight-gold", "Base de données Gold")


# ======================================================================================
# 2. FONCTION DE TRANSFORMATION SPÉCIFIQUE
# ======================================================================================

def transform_to_dim_customer(silver_df: DataFrame) -> DataFrame:
    """
    Crée la dimension Client/Géographie à partir des données Silver.
    NOTE : C'est une approche SCD Type 1 - Elle écrase les données du client s'il y a un changement.
    """
    logging.info("Début de la création de la dimension 'DimCustomer'.")
    
    # a. Isoler les colonnes et dédupliquer les clients
    # DÉFI : Que se passe-t-il si un client a deux adresses différentes dans les données sources ?
    # .distinct() n'est pas une solution robuste pour définir un client unique.
    df_dim_customer = silver_df.select(
        "CUSTOMERNAME", 
        "CONTACTLASTNAME", 
        "CONTACTFIRSTNAME", 
        "PHONE", 
        "ADDRESSLINE1", 
        "CITY", 
        "STATE", 
        "POSTALCODE", 
        "COUNTRY", 
        "TERRITORY"
    ).distinct()

    # b. Ajouter une clé de substitution
    df_dim_customer = df_dim_customer.withColumn("CUSTOMERKEY", monotonically_increasing_id())

    # c. Réorganiser les colonnes
    df_dim_customer_final = df_dim_customer.select(
        "CUSTOMERKEY", 
        "CUSTOMERNAME", 
        "CONTACTLASTNAME", 
        "CONTACTFIRSTNAME", 
        "PHONE", 
        "ADDRESSLINE1", 
        "CITY", 
        "STATE", 
        "POSTALCODE", 
        "COUNTRY", 
        "TERRITORY"
    )
    
    logging.info("Transformation vers 'DimCustomer' terminée.")
    return df_dim_customer_final

# ======================================================================================
# 3. POINT D'ENTRÉE PRINCIPAL (MAIN)
# ======================================================================================
if __name__ == "__main__":
    
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    logging.info("===================================================")
    logging.info("DÉMARRAGE DU PIPELINE SILVER-TO-GOLD (DimCustomer)")
    logging.info("===================================================")
    
    try:
        # Récupération des paramètres
        storage_account = dbutils.widgets.get("storage_account").strip()
        container = dbutils.widgets.get("container").strip()
        silver_folder = dbutils.widgets.get("silver_folder").strip()
        secret_scope = dbutils.widgets.get("secret_scope").strip()
        adls_secret_key = dbutils.widgets.get("adls_secret_key").strip()
        jdbc_hostname = dbutils.widgets.get("jdbc_hostname").strip()
        jdbc_database = dbutils.widgets.get("jdbc_database").strip()
        sql_user_key = dbutils.widgets.get("sql_user_key").strip()
        sql_password_key = dbutils.widgets.get("sql_password_key").strip()

        # --- ORCHESTRATION ---
        
        # 1. Utiliser la bibliothèque pour configurer les accès
        common_utils.setup_adls_access(spark, dbutils, storage_account, secret_scope, adls_secret_key)
        jdbc_url, connection_props = common_utils.get_jdbc_connection_properties(
            dbutils, jdbc_hostname, jdbc_database, secret_scope, sql_user_key, sql_password_key
        )
        
        # 2. Utiliser la bibliothèque pour lire les données
        source_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/{silver_folder}"
        silver_dataframe = common_utils.read_silver_data(spark, source_path)
        
        # 3. Exécuter la logique de transformation spécifique à ce notebook
        dim_customer_dataframe = transform_to_dim_customer(silver_dataframe)
        
        # 4. Utiliser la bibliothèque pour écrire le résultat
        common_utils.write_dimension_to_gold(dim_customer_dataframe, "DimCustomer", jdbc_url, connection_props)
        
        logging.info("===================================================")
        logging.info("PIPELINE SILVER-TO-GOLD (DimCustomer) TERMINÉ AVEC SUCCÈS")
        logging.info("===================================================")

    except Exception as e:
        logging.error("Le pipeline a échoué dans le bloc principal.", exc_info=True)
        raise e